In [1]:
###################
#
# Building a Cifar10 classifier using the AlexNet Architecture
#
# Will use the low level APIs of tensorflow
#
#####################

In [2]:
import tensorflow as tf
import numpy as np
import pickle
from sklearn.metrics import accuracy_score
import random as rn
from tensorflow import set_random_seed


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#np.random.seed(2017)
#rn.seed(1254)
#tf.set_random_seed(1234)

In [17]:
#Constants
trainingFiles = ["D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\data_batch_1",
                "D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\data_batch_2",
                "D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\data_batch_3",
                "D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\data_batch_4",
                "D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\data_batch_5"]

testFile = ['D:/Anurag/CIFAR10_DATA/cifar-10-batches-py/test_batch']

cifarClassesFile = 'D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\batches.meta'#.txt'

saveModelPath = "D:/Anurag/watermarking/cifar10code/alexNet_cifar10_trainedModel/run5_LR_1e-4_WD_1e-3"

_NUM_TRAINING_IMAGES = 50000
_NUM_TEST_IMAGES = 10000

_HEIGHT = 32
_WIDTH = 32
_CHANNELS = 3

_IMAGE_BYTES = _HEIGHT * _WIDTH * _CHANNELS + 1 # + 1 for the label

_NUM_CLASSES = 10

_SHUFFLE_BUFFER = 20000

_SEED=7

_BATCH_SIZE=128
_NUM_EPOCHS = 100# 200
_LEARNING_RATE = 0.0001
_WEIGHT_DECAY = 0.001

IS_TRAINING = True # used for dropout

_TENSORBOARD_TRAIN_LOGDIR = 'D:/Anurag/watermarking/cifar10code/tensorboard_logdir'

_SAVE_MODEL_EVERY_NUM_EPOCHS = 5

In [18]:
TrainingImages = np.zeros(shape=(_NUM_TRAINING_IMAGES, _HEIGHT, _WIDTH, _CHANNELS), dtype=float)
TrainingLabels = np.zeros(shape=(_NUM_TRAINING_IMAGES), dtype=int)

TestImages = np.zeros(shape=(_NUM_TEST_IMAGES,_HEIGHT, _WIDTH, _CHANNELS ), dtype=float)
TestLabels = np.zeros(shape=(_NUM_TEST_IMAGES), dtype=int)


In [19]:
begin=0
for filename in trainingFiles:
    print('Reading file:', filename)
    with open(filename, mode='rb') as f:
        data=pickle.load(f, encoding='bytes')
        raw_images = data[b'data']
        labels = data[b'labels']
        images = raw_images.reshape([-1, _CHANNELS, _HEIGHT, _WIDTH])
        images = images.transpose([0, 2, 3, 1])
        
        print('extracted num of images:', len(images))
        end = begin + len(images)
        TrainingImages[begin: end,:] = images
        TrainingLabels[begin: end] = labels
        begin += len(images)

Reading file: D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\data_batch_1
extracted num of images: 10000
Reading file: D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\data_batch_2
extracted num of images: 10000
Reading file: D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\data_batch_3
extracted num of images: 10000
Reading file: D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\data_batch_4
extracted num of images: 10000
Reading file: D:\Anurag\CIFAR10_DATA\cifar-10-batches-py\data_batch_5
extracted num of images: 10000


In [20]:
TrainingImages = TrainingImages / 255

In [21]:
#Shuffle the training data
TrainingImagesRandom = np.zeros(shape=(_NUM_TRAINING_IMAGES, _HEIGHT, _WIDTH, _CHANNELS), dtype=float)
TrainingLabelsRandom = np.zeros(shape=(_NUM_TRAINING_IMAGES), dtype=int)

randomOrder = np.arange(start=0, stop=len(TrainingImages), dtype=int)
print(randomOrder)
print(TrainingLabels)
np.random.shuffle(randomOrder)
print(randomOrder)

for i in range(_NUM_TRAINING_IMAGES):
    TrainingImagesRandom[i,:,:,:] = TrainingImages[randomOrder[i], :, :, :]
    TrainingLabelsRandom[i] = TrainingLabels[randomOrder[i]]

#print("len of randomOrder:", len(randomOrder))
print("len of TrainingImages:", len(TrainingLabels))
#let's check the suffle is correct. We know orig 0th label = 6.

for i in range(_NUM_TRAINING_IMAGES):
    if (randomOrder[i] == 0):
        print("i:", i, "label:", TrainingLabels[0])
        print("Original Label:", TrainingLabelsRandom[i])

TrainingImages = TrainingImagesRandom
TrainingLabels = TrainingLabelsRandom

[    0     1     2 ... 49997 49998 49999]
[6 9 9 ... 9 1 1]
[25933  3775 32980 ...   901 40045 27059]
len of TrainingImages: 50000
i: 2469 label: 6
Original Label: 6


In [22]:
#create a one-hot encodong for the labels
TrainingLabels_oneHot = np.zeros(shape=(len(TrainingLabels), _NUM_CLASSES))
for i in range(len(TrainingLabels)):
    TrainingLabels_oneHot[i, TrainingLabels[i]] = 1

In [23]:
begin=0
for filename in testFile:
    print('Reading file:', filename)
    with open(filename,'rb') as f:
        data = pickle.load(f, encoding='bytes')
        raw_images = data[b'data']
        labels = data[b'labels']
        images = raw_images.reshape([-1, _CHANNELS, _HEIGHT, _WIDTH])
        images = images.transpose([0, 2, 3, 1])
        #images /= 255
        print('extracted num of images:', len(images))
        end = begin + len(images)
        TestImages[begin: end,:] = images
        TestLabels[begin: end] = labels
        begin += len(images)

TestImages = TestImages / 255

Reading file: D:/Anurag/CIFAR10_DATA/cifar-10-batches-py/test_batch
extracted num of images: 10000


In [24]:
#Convert labels into one-hot
TestLabels_oneHot = np.zeros(shape=(len(TestLabels), _NUM_CLASSES))
for i in range(len(TestLabels)):
    TestLabels_oneHot[i, TestLabels[i]] = 1

In [26]:
tf.reset_default_graph()

X_train = tf.placeholder(tf.float32, shape=(None, _HEIGHT, _WIDTH, _CHANNELS), name='input_Xtrain')
Y_train = tf.placeholder(tf.int32, shape=(None, 10), name='inputLabel')
isTraining = tf.placeholder(tf.bool, name='isTrainingBool')

#building the AlexNet model
conv1=tf.layers.conv2d(inputs= X_train, filters=96, kernel_size=(4,4), strides=(1,1), padding='same', activation=tf.nn.relu, name='conv1') # (128, 32, 32, 96)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2) # (128, 16, 16, 96)
norm1 = tf.layers.batch_normalization(inputs=pool1, training=isTraining, name='norm1', momentum=0.1) # (128, 16, 16, 96)
#norm1 = tf.contrib.layers.batch_norm(inputs=pool1, is_training=isTraining, updates_collections=None, name='norm1')

conv2 = tf.layers.conv2d(inputs=norm1, filters=256, kernel_size=(4,4), strides=(1,1), padding='same', activation=tf.nn.relu, name='conv2') # (128, 16, 16, 256)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2) # (128, 8, 8, 256)
norm2 = tf.layers.batch_normalization(inputs=pool2, training=isTraining, name='norm2', momentum=0.1) # (128, 8 , 8, 256)
#norm2 = tf.contrib.layers.batch_norm(inputs=pool2, is_training=isTraining, updates_collections=None, name='norm2')

conv3 = tf.layers.conv2d(inputs=norm2, filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation=tf.nn.relu, name='conv3') # (128, 8, 8, 384)
conv4 = tf.layers.conv2d(inputs=conv3, filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation=tf.nn.relu, name='conv4') # (128, 8, 8, 384)
conv5 = tf.layers.conv2d(inputs=conv4, filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation=tf.nn.relu, name='conv5') # (128, 8, 8, 384)

pool3= tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2) # (128, 4, 4, 384)

drop1 = tf.layers.dropout(inputs=pool3, rate=0.5, training=isTraining) #(128, 4, 4, 384)

flatten1 = tf.reshape(drop1, [-1, 4 * 4 * 384])
fc6 = tf.layers.dense(inputs=flatten1, units=4096, name='fc6') #(128, 4096)
drop2 = tf.layers.dropout(inputs=fc6, rate=0.5, training=isTraining) #(128, 4096)
fc7= tf.layers.dense(inputs=drop2, units=4096, name='fc7') #(128, 4096)

logits = tf.layers.dense(inputs=fc7, units=_NUM_CLASSES, name='logits_output',use_bias=False ) #(128, 10) #maybe the name should be different from the variable name?


In [27]:
outProb = tf.nn.softmax(logits, name='softmax_output')

In [28]:
predClass = tf.argmax(outProb, axis=1)
predClass = tf.reshape(predClass, shape=[-1])
actualClass = tf.argmax(Y_train, axis=1) # this is to reduce from one-hot to actual labels
actualClass = tf.reshape(actualClass, shape=[-1])

In [29]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=Y_train, logits=logits) + _WEIGHT_DECAY * tf.add_n( [tf.nn.l2_loss(v) for v in tf.trainable_variables()])

tf.summary.scalar(tensor=loss, name='loss')

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_step = tf.train.AdamOptimizer(learning_rate=_LEARNING_RATE).minimize(loss)

In [30]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    tensorboard_training_writer = tf.summary.FileWriter(_TENSORBOARD_TRAIN_LOGDIR, sess.graph)
    all_summary_ops = tf.summary.merge_all()
    
    for e in range(_NUM_EPOCHS):
        #Pull out batches of training data
        print('starting training Epoch:', e, ' of ', _NUM_EPOCHS )
        IS_TRAINING = True
        totalLoss = 0
        totalAccuracy = 0
        numSteps = len(TrainingImages)//_BATCH_SIZE #note // does integer division

        #let's reset the training metrics local variables.
        #accuracyMetric_variables = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="trainAccuracy")
        #sess.run(tf.variables_initializer(var_list=accuracyMetric_variables))
        
        

        for step in range(numSteps):
            begin = step * _BATCH_SIZE
            end = begin + _BATCH_SIZE
            trainBatch_X = TrainingImages[begin: end, :, :, :]
            trainBatch_Y = TrainingLabels_oneHot[begin: end, :]
            
            #We now perform the optimisation
            _, lossBatch, actBatch, predBatch, summaryValues = sess.run(
                [train_step, loss, actualClass, predClass, all_summary_ops], 
                feed_dict={X_train: trainBatch_X, Y_train: trainBatch_Y, isTraining: IS_TRAINING})
            #accuracyBatch = sess.run(accuracy)
            totalLoss += lossBatch
            
            accBatch = accuracy_score(y_true=actBatch, y_pred=predBatch)
            totalAccuracy += accBatch
            if step % 50 == 0:
                print('got a loss of (batch is ', len(trainBatch_Y),') :', lossBatch)
                print('got a batch accuracy of:', accBatch)
                print('value of actuals', actBatch)
                print('value of predictions:', predBatch)
                #tensorboard
                tensorboard_training_writer.add_summary(summaryValues, step)
            #
            #print('got batch accuracy of:', accuracyBatch)
            #print('shape of predictions', predBatch.shape)
            #print('value of predictions:', predBatch)
            #print('value of actuals', actBatch)
            #print('shape of accuracy:', accuracyBatch.shape)
            #print('value of accuracy:', accuracyBatch)
            
            
        if end < len(TrainingImages):
            trainBatch_X = TrainingImages[end: len(TrainingImages), :, :, :]
            trainBatch_Y = TrainingLabels_oneHot[end: len(TrainingImages), :]
            _, lossBatch, actBatch, predBatch = sess.run([train_step, loss, actualClass, predClass], feed_dict={X_train: trainBatch_X, Y_train: trainBatch_Y, isTraining: IS_TRAINING})
            totalLoss += lossBatch
            accBatch = accuracy_score(y_true=actBatch, y_pred=predBatch)
            totalAccuracy += accBatch
            numSteps += 1
            #print('got a loss of (batch is ', len(trainBatch_Y),') :', lossBatch)
                  
        #end of 1 epoch. Do validation here
        print('Completed Epoch. Training accuracy:', totalAccuracy / (numSteps), 'Total loss:', totalLoss)
        IS_TRAINING = False
        totalValLoss=0
        totalValAccuracy = 0
        numSteps = len(TestImages)//_BATCH_SIZE
        for i in range(numSteps):
            begin = i * _BATCH_SIZE
            end = begin + _BATCH_SIZE
            testBatch_X = TestImages[begin:end,:,:,:]
            testBatch_Y = TestLabels_oneHot[begin:end,:]
            
            lossBatch, actBatch, predBatch = sess.run([loss, actualClass, predClass], feed_dict={X_train: testBatch_X, Y_train: testBatch_Y, isTraining: IS_TRAINING})
            #print('got values:', lossBatch, actBatch, predBatch )
            #print('----- again -----')
            #lossBatch, accuracyBatch, actBatch, predBatch = sess.run([loss, accuracy, actualClass, predClass], feed_dict={X_train: testBatch_X, Y_train: testBatch_Y, isTraining: IS_TRAINING})
            #print('got values:', lossBatch, accuracyBatch, actBatch, predBatch )
            #print('----- ???????????? -----')

            totalValLoss += lossBatch
            accBatch = accuracy_score(y_true=actBatch, y_pred=predBatch)
            totalValAccuracy += accBatch
            
            if i % 50 == 0:
                print('Actual Class:', actBatch)
                print('Predicted Class:', predBatch)
            
        if end < len(TestImages):
            begin = end
            end = len(TestImages)
            testBatch_X = TestImages[begin:end,:,:,:]
            testBatch_Y = TestLabels_oneHot[begin:end,:]
            lossBatch, actBatch, predBatch = sess.run([loss, actualClass, predClass], feed_dict={X_train: testBatch_X, Y_train: testBatch_Y, isTraining: IS_TRAINING})
            totalValLoss += lossBatch
            accBatch = accuracy_score(y_true=actBatch, y_pred=predBatch)
            totalValAccuracy += accBatch
            numSteps+=1
            
        #completed validation
        print('Completed validation. Validation accuracy:', totalValAccuracy/numSteps)
        
        #Learning rate decay
        if ((e+1) % 50 == 0):
            _LEARNING_RATE = _LEARNING_RATE / 10
            print('reducing learning rate to:', _LEARNING_RATE)
            
        if ((e+1) % _SAVE_MODEL_EVERY_NUM_EPOCHS == 0):
            saveModelPathWithEpochNum = saveModelPath + '_epoch_'+ str(e+1)
            print("saving Model")
            tf.saved_model.simple_save(sess, export_dir=saveModelPathWithEpochNum, 
                                       inputs={"inputImageBatch": X_train, "inputClassBatch": Y_train, "isTrainingBool":isTraining},
                                       outputs={"predictedClassBatch":predClass})
            print('Completed saving Model to path:', saveModelPath)
    
    
    #completed training for all Epochs.
    
    print('Finis')

starting training Epoch: 0  of  100
got a loss of (batch is  128 ) : 8.135054
got a batch accuracy of: 0.1484375
value of actuals [3 6 3 4 7 4 6 6 1 9 8 6 0 6 0 6 5 0 2 8 5 3 6 9 6 4 6 9 9 4 4 8 6 6 5 1 6
 0 6 2 9 6 2 3 1 8 4 4 9 9 2 0 7 5 9 9 2 2 2 4 6 5 1 8 4 6 2 4 3 4 3 5 1 6
 8 1 9 5 5 7 7 9 2 8 8 0 1 8 6 2 5 4 6 1 2 6 9 0 8 0 4 3 3 9 0 7 5 6 0 7 8
 9 5 3 8 9 5 6 8 7 2 7 4 2 4 2 3 1]
value of predictions: [6 0 0 4 6 4 4 4 3 8 5 0 8 5 0 4 5 5 0 4 7 4 5 6 6 5 8 4 0 4 7 9 0 6 7 4 0
 7 4 5 0 6 0 1 3 8 4 5 1 4 6 5 8 8 6 5 4 5 6 9 4 8 5 4 8 4 6 4 5 7 5 8 5 5
 8 8 4 7 4 1 0 1 0 8 4 8 8 8 6 7 6 6 0 4 7 4 4 8 6 0 8 4 8 2 6 6 5 8 7 4 4
 5 4 4 4 4 4 6 0 6 4 6 4 0 6 6 5 0]
got a loss of (batch is  128 ) : 6.9264445
got a batch accuracy of: 0.3671875
value of actuals [1 7 2 3 4 1 2 0 1 4 9 4 9 8 7 3 7 8 4 4 5 0 4 7 0 2 0 3 8 7 6 3 6 0 7 0 9
 2 3 7 0 9 8 6 4 7 1 9 6 9 2 6 9 2 7 6 8 8 1 5 1 4 2 4 2 1 0 4 4 3 8 2 9 3
 1 5 9 0 4 0 7 2 2 7 5 1 3 3 5 2 4 7 9 3 2 7 2 4 3 3 1 9 4 1 2 7 3 0 9 2 0
 4 9 8

got a loss of (batch is  128 ) : 5.0802226
got a batch accuracy of: 0.6875
value of actuals [2 4 9 3 9 8 2 2 2 5 6 2 4 7 2 7 7 0 0 8 2 9 8 1 5 0 2 5 3 9 9 2 7 6 7 2 6
 2 8 3 3 5 8 3 7 1 1 2 1 2 2 6 7 8 9 7 0 6 8 7 7 1 5 3 5 8 1 9 8 3 4 3 5 4
 9 2 7 2 9 9 8 9 8 2 9 3 7 9 1 7 6 7 6 3 6 4 8 9 5 2 5 4 2 6 6 7 0 0 0 7 2
 0 3 3 4 4 1 8 9 0 1 2 3 2 4 4 8 9]
value of predictions: [2 4 9 7 3 8 5 2 5 5 3 5 4 7 5 7 5 8 8 8 4 9 8 1 5 0 2 5 3 9 9 2 7 9 7 2 6
 2 8 3 1 5 1 3 7 1 1 7 1 2 2 6 7 8 9 7 0 7 8 7 7 1 5 3 5 0 1 9 8 4 4 3 3 4
 9 7 7 0 9 1 8 9 8 2 1 3 7 9 1 7 6 7 0 5 6 4 8 9 2 4 5 4 2 9 6 7 0 0 8 4 3
 4 2 5 4 7 1 6 9 0 9 2 3 4 3 4 8 7]
got a loss of (batch is  128 ) : 5.056713
got a batch accuracy of: 0.6171875
value of actuals [2 7 8 9 2 6 0 1 4 7 7 0 1 0 5 5 4 7 2 1 8 8 6 5 4 3 6 0 0 0 6 6 9 0 6 2 3
 6 4 3 8 4 9 2 3 7 9 1 4 2 4 3 8 4 3 0 0 7 0 2 2 9 5 3 1 9 3 6 1 0 0 2 4 9
 3 0 0 2 0 9 4 2 9 1 1 0 1 2 1 5 1 4 7 9 4 9 7 4 0 3 4 1 1 8 5 2 4 5 2 9 1
 8 6 5 6 5 3 9 2 6 6 6 3 6 6 2 2 6]
value of 

got a loss of (batch is  128 ) : 3.8588223
got a batch accuracy of: 0.7578125
value of actuals [5 9 7 3 9 0 9 6 6 5 0 2 2 6 9 4 9 4 0 7 0 0 8 6 7 6 3 4 8 7 5 1 3 8 5 6 6
 2 4 1 8 7 4 4 3 6 4 4 6 6 9 2 1 2 7 8 6 6 4 6 4 7 6 8 4 3 3 7 8 3 2 1 8 0
 5 6 7 4 1 6 0 9 5 4 7 0 8 7 2 8 3 9 4 5 1 8 5 2 5 0 1 6 1 2 4 4 7 0 4 5 2
 4 8 7 2 1 5 7 3 0 4 4 9 4 4 6 7 4]
value of predictions: [2 9 5 3 9 0 9 6 6 5 0 2 2 5 9 4 9 4 0 3 0 9 8 6 2 6 5 4 8 7 3 1 5 8 5 6 6
 2 4 1 8 7 4 4 4 6 4 4 6 6 9 4 1 2 7 8 6 6 4 6 2 4 4 8 4 3 3 7 8 3 2 1 8 0
 5 3 7 2 1 6 0 1 5 4 7 2 8 7 2 8 3 9 4 5 9 8 4 2 3 0 4 6 1 2 4 7 7 2 4 7 8
 9 8 9 8 1 2 3 3 0 4 4 9 4 4 6 7 4]
got a loss of (batch is  128 ) : 3.8272958
got a batch accuracy of: 0.7734375
value of actuals [3 2 6 4 1 3 5 5 4 1 6 5 6 9 6 3 3 1 2 8 7 2 8 1 8 6 4 2 6 3 7 1 7 3 5 3 4
 1 9 1 9 4 4 3 4 4 8 5 8 0 9 8 1 4 8 3 2 9 0 8 3 7 4 9 5 2 7 1 9 4 2 6 1 8
 4 8 2 8 0 3 7 4 2 0 6 2 7 0 5 8 8 3 9 9 7 8 9 4 1 8 3 4 8 1 3 6 8 8 9 8 0
 3 9 4 9 6 0 0 6 7 8 1 1 8 0 5 3 3]
value

Actual Class: [0 4 8 4 7 1 3 2 0 9 1 8 8 6 8 1 7 1 0 9 7 6 4 5 8 2 1 2 5 8 1 1 8 8 0 9 0
 0 4 3 3 5 6 6 3 1 4 4 7 8 0 1 4 8 4 6 2 2 2 6 3 4 0 8 1 3 8 3 8 9 1 6 1 9
 8 2 0 4 7 4 0 0 7 6 8 9 8 6 2 8 8 0 1 1 0 7 7 4 4 7 7 2 6 6 7 4 5 4 0 9 4
 7 2 5 3 2 2 7 6 2 6 4 9 6 8 3 8 5]
Predicted Class: [0 2 8 4 7 0 9 2 0 9 1 8 0 6 8 1 7 1 0 8 7 6 3 4 0 2 1 2 2 8 1 1 8 8 8 9 0
 0 4 4 3 3 6 6 3 1 2 4 5 8 0 1 4 8 2 6 2 2 2 6 3 2 0 8 1 3 3 3 8 1 1 6 1 9
 0 2 0 4 4 4 0 0 7 6 8 9 8 6 2 8 0 2 1 0 2 7 5 2 4 7 0 2 3 6 4 4 5 2 0 1 9
 7 2 5 2 2 2 5 6 2 6 4 1 4 8 3 8 5]
Completed validation. Validation accuracy: 0.7152887658227848
starting training Epoch: 4  of  100
got a loss of (batch is  128 ) : 3.2082572
got a batch accuracy of: 0.6875
value of actuals [3 6 3 4 7 4 6 6 1 9 8 6 0 6 0 6 5 0 2 8 5 3 6 9 6 4 6 9 9 4 4 8 6 6 5 1 6
 0 6 2 9 6 2 3 1 8 4 4 9 9 2 0 7 5 9 9 2 2 2 4 6 5 1 8 4 6 2 4 3 4 3 5 1 6
 8 1 9 5 5 7 7 9 2 8 8 0 1 8 6 2 5 4 6 1 2 6 9 0 8 0 4 3 3 9 0 7 5 6 0 7 8
 9 5 3 8 9 5 6 8 7 2 7 4 2 4 2 3

got a loss of (batch is  128 ) : 2.4833684
got a batch accuracy of: 0.8125
value of actuals [1 7 2 3 4 1 2 0 1 4 9 4 9 8 7 3 7 8 4 4 5 0 4 7 0 2 0 3 8 7 6 3 6 0 7 0 9
 2 3 7 0 9 8 6 4 7 1 9 6 9 2 6 9 2 7 6 8 8 1 5 1 4 2 4 2 1 0 4 4 3 8 2 9 3
 1 5 9 0 4 0 7 2 2 7 5 1 3 3 5 2 4 7 9 3 2 7 2 4 3 3 1 9 4 1 2 7 3 0 9 2 0
 4 9 8 4 6 8 7 7 2 2 4 8 2 9 2 4 7]
value of predictions: [1 7 2 3 4 1 2 2 1 4 9 4 9 8 7 3 5 8 4 4 3 0 4 7 0 2 2 3 8 7 6 5 3 1 7 9 9
 2 3 7 0 9 8 6 8 7 1 9 6 9 2 6 9 2 5 6 8 8 9 5 1 4 2 4 4 1 0 5 4 2 8 4 9 3
 9 5 9 0 4 0 7 2 2 7 5 9 3 3 7 2 4 7 9 3 2 7 2 4 3 8 1 9 4 7 2 7 3 0 9 2 0
 3 9 8 4 6 8 7 7 2 6 7 8 2 9 2 6 7]
got a loss of (batch is  128 ) : 2.3011312
got a batch accuracy of: 0.828125
value of actuals [9 0 1 4 9 9 3 8 3 3 9 9 8 7 0 4 9 4 9 5 8 3 6 9 3 4 6 9 1 3 5 6 2 6 1 7 9
 6 7 0 0 1 6 1 8 0 2 4 7 1 3 4 7 1 1 7 8 7 1 9 9 5 7 4 7 7 9 7 3 7 1 1 2 8
 4 0 7 2 0 9 4 6 7 9 5 3 3 8 5 6 1 8 8 2 7 1 9 1 1 7 5 4 8 0 0 9 1 0 8 6 2
 0 3 6 2 9 9 8 0 8 0 9 2 2 1 6 7 8]
value of 

...

Completed Epoch. Training accuracy: 0.9898297634271099 Total loss: 118.83714696764946
Actual Class: [3 8 8 0 6 6 1 6 3 1 0 9 5 7 9 8 5 7 8 6 7 0 4 9 5 2 4 0 9 6 6 5 4 5 9 2 4
 1 9 5 4 6 5 6 0 9 3 9 7 6 9 8 0 3 8 8 7 7 4 6 7 3 6 3 6 2 1 2 3 7 2 6 8 8
 0 2 9 3 3 8 8 1 1 7 2 5 2 7 8 9 0 3 8 6 4 6 6 0 0 7 4 5 6 3 1 1 3 6 8 7 4
 0 6 2 1 3 0 4 2 7 8 3 1 2 8 0 8 3]
Predicted Class: [3 8 8 0 6 6 1 2 3 1 0 9 5 7 9 0 5 7 8 6 7 0 4 9 5 2 5 0 9 6 6 5 2 5 9 1 4
 1 9 5 4 6 5 6 0 9 3 9 7 2 9 8 2 5 8 8 7 5 3 4 7 5 0 0 6 2 1 0 3 7 2 6 8 8
 0 2 0 3 3 8 8 1 1 7 2 5 2 0 8 9 0 3 8 6 4 6 6 0 0 7 4 5 6 6 1 1 3 6 8 7 4
 0 2 2 1 0 0 4 6 7 8 3 1 2 8 0 8 3]
Actual Class: [0 4 8 4 7 1 3 2 0 9 1 8 8 6 8 1 7 1 0 9 7 6 4 5 8 2 1 2 5 8 1 1 8 8 0 9 0
 0 4 3 3 5 6 6 3 1 4 4 7 8 0 1 4 8 4 6 2 2 2 6 3 4 0 8 1 3 8 3 8 9 1 6 1 9
 8 2 0 4 7 4 0 0 7 6 8 9 8 6 2 8 8 0 1 1 0 7 7 4 4 7 7 2 6 6 7 4 5 4 0 9 4
 7 2 5 3 2 2 7 6 2 6 4 9 6 8 3 8 5]
Predicted Class: [0 7 8 4 7 1 3 2 0 9 1 8 0 6 8 1 7 1 0 8 7 6 4 4 8 0 1 2 5 8 1 1 8 8 

got a loss of (batch is  128 ) : 0.29263172
got a batch accuracy of: 0.9921875
value of actuals [1 7 2 3 4 1 2 0 1 4 9 4 9 8 7 3 7 8 4 4 5 0 4 7 0 2 0 3 8 7 6 3 6 0 7 0 9
 2 3 7 0 9 8 6 4 7 1 9 6 9 2 6 9 2 7 6 8 8 1 5 1 4 2 4 2 1 0 4 4 3 8 2 9 3
 1 5 9 0 4 0 7 2 2 7 5 1 3 3 5 2 4 7 9 3 2 7 2 4 3 3 1 9 4 1 2 7 3 0 9 2 0
 4 9 8 4 6 8 7 7 2 2 4 8 2 9 2 4 7]
value of predictions: [1 7 2 3 4 1 2 0 1 4 9 4 9 8 7 3 7 8 4 4 5 0 4 7 0 2 0 3 8 7 6 3 6 0 7 0 9
 2 3 7 0 9 8 6 4 7 1 9 6 9 2 6 9 2 7 6 8 8 1 5 1 4 2 4 2 1 0 4 4 3 9 2 9 3
 1 5 9 0 4 0 7 2 2 7 5 1 3 3 5 2 4 7 9 3 2 7 2 4 3 3 1 9 4 1 2 7 3 0 9 2 0
 4 9 8 4 6 8 7 7 2 2 4 8 2 9 2 4 7]
got a loss of (batch is  128 ) : 0.28664538
got a batch accuracy of: 0.9921875
value of actuals [9 0 1 4 9 9 3 8 3 3 9 9 8 7 0 4 9 4 9 5 8 3 6 9 3 4 6 9 1 3 5 6 2 6 1 7 9
 6 7 0 0 1 6 1 8 0 2 4 7 1 3 4 7 1 1 7 8 7 1 9 9 5 7 4 7 7 9 7 3 7 1 1 2 8
 4 0 7 2 0 9 4 6 7 9 5 3 3 8 5 6 1 8 8 2 7 1 9 1 1 7 5 4 8 0 0 9 1 0 8 6 2
 0 3 6 2 9 9 8 0 8 0 9 2 2 1 6 7 8]
val

got a loss of (batch is  128 ) : 0.26659915
got a batch accuracy of: 1.0
value of actuals [2 7 8 9 2 6 0 1 4 7 7 0 1 0 5 5 4 7 2 1 8 8 6 5 4 3 6 0 0 0 6 6 9 0 6 2 3
 6 4 3 8 4 9 2 3 7 9 1 4 2 4 3 8 4 3 0 0 7 0 2 2 9 5 3 1 9 3 6 1 0 0 2 4 9
 3 0 0 2 0 9 4 2 9 1 1 0 1 2 1 5 1 4 7 9 4 9 7 4 0 3 4 1 1 8 5 2 4 5 2 9 1
 8 6 5 6 5 3 9 2 6 6 6 3 6 6 2 2 6]
value of predictions: [2 7 8 9 2 6 0 1 4 7 7 0 1 0 5 5 4 7 2 1 8 8 6 5 4 3 6 0 0 0 6 6 9 0 6 2 3
 6 4 3 8 4 9 2 3 7 9 1 4 2 4 3 8 4 3 0 0 7 0 2 2 9 5 3 1 9 3 6 1 0 0 2 4 9
 3 0 0 2 0 9 4 2 9 1 1 0 1 2 1 5 1 4 7 9 4 9 7 4 0 3 4 1 1 8 5 2 4 5 2 9 1
 8 6 5 6 5 3 9 2 6 6 6 3 6 6 2 2 6]
got a loss of (batch is  128 ) : 0.2761272
got a batch accuracy of: 0.9921875
value of actuals [0 0 9 7 8 1 6 5 5 0 7 8 5 8 0 7 7 0 6 0 3 1 4 0 2 3 0 3 5 3 0 8 1 9 0 2 8
 1 6 5 3 7 4 8 4 2 2 4 1 1 3 4 6 0 0 0 0 9 2 6 0 8 9 6 8 9 3 8 5 2 3 6 9 6
 2 6 4 7 8 8 3 9 7 5 3 3 5 0 7 1 8 2 0 9 4 1 4 0 1 0 1 0 2 0 1 5 1 0 4 5 2
 4 4 7 1 3 5 3 3 9 2 5 7 7 7 9 1 5]
value of p

got a loss of (batch is  128 ) : 0.27461147
got a batch accuracy of: 0.9921875
value of actuals [3 2 6 4 1 3 5 5 4 1 6 5 6 9 6 3 3 1 2 8 7 2 8 1 8 6 4 2 6 3 7 1 7 3 5 3 4
 1 9 1 9 4 4 3 4 4 8 5 8 0 9 8 1 4 8 3 2 9 0 8 3 7 4 9 5 2 7 1 9 4 2 6 1 8
 4 8 2 8 0 3 7 4 2 0 6 2 7 0 5 8 8 3 9 9 7 8 9 4 1 8 3 4 8 1 3 6 8 8 9 8 0
 3 9 4 9 6 0 0 6 7 8 1 1 8 0 5 3 3]
value of predictions: [3 2 6 4 1 3 5 5 4 1 6 5 6 9 6 3 3 1 2 8 7 2 8 1 8 6 4 2 6 3 7 1 7 3 5 3 4
 1 9 1 9 4 4 3 4 4 8 5 8 0 9 8 1 7 8 3 2 9 0 8 3 7 4 9 5 2 7 1 9 4 2 6 1 8
 4 8 2 8 0 3 7 4 2 0 6 2 7 0 5 8 8 3 9 9 7 8 9 4 1 8 3 4 8 1 3 6 8 8 9 8 0
 3 9 4 9 6 0 0 6 7 8 1 1 8 0 5 3 3]
Completed Epoch. Training accuracy: 0.9917679028132992 Total loss: 112.06735235452652
Actual Class: [3 8 8 0 6 6 1 6 3 1 0 9 5 7 9 8 5 7 8 6 7 0 4 9 5 2 4 0 9 6 6 5 4 5 9 2 4
 1 9 5 4 6 5 6 0 9 3 9 7 6 9 8 0 3 8 8 7 7 4 6 7 3 6 3 6 2 1 2 3 7 2 6 8 8
 0 2 9 3 3 8 8 1 1 7 2 5 2 7 8 9 0 3 8 6 4 6 6 0 0 7 4 5 6 3 1 1 3 6 8 7 4
 0 6 2 1 3 0 4 2 7 8 3 1 2 8 0 8 3]

Completed validation. Validation accuracy: 0.8391020569620253
reducing learning rate to: 1.0000000000000002e-06
saving Model
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: D:/Anurag/watermarking/cifar10code/alexNet_cifar10_trainedModel/run5_LR_1e-4_WD_1e-3_epoch_100\saved_model.pb
Completed saving Model to path: D:/Anurag/watermarking/cifar10code/alexNet_cifar10_trainedModel/run5_LR_1e-4_WD_1e-3
Finis
